# Credit Risk Evaluator

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

## Retrieve the Data

The data is located in the Challenge Files Folder:

* `lending_data.csv`

Import the data using Pandas. Display the resulting dataframe to confirm the import was successful.

In [3]:
# Import the data
df = pd.read_csv('Resources/lending_data.csv')
df

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1


In [4]:
# Drop duplicate data
df = df.drop_duplicates()

df

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
...,...,...,...,...,...,...,...,...
77524,16900.0,10.302,77500,0.612903,10,2,47500,1
77526,18300.0,10.895,83100,0.638989,11,2,53100,1
77528,15100.0,9.557,70500,0.574468,9,2,40500,1
77531,19100.0,11.261,86600,0.653580,12,2,56600,1


## Predict Model Performance

This process will run two models on this data: a Logistic Regression, and a Random Forests Classifier. 

At first glance, the data has 6 features, all data types are numeric. However some columns have a linear relationship between them(`['debt_to_income']=['total_debt']/['borrower_income']`) and also `['number_of_accounts']` seems not a factor to verify a person's payback ability. Therefore, the number of explanatory variables is 4 ,and the number of noise variables is 2.

In my opinion, the performance of Logistic Regression's prediction could be better as Random Forest Classifier would performance better when the data set has many variables. 

## Split the Data into Training and Testing Sets

In [5]:
y = df['loan_status']
ycheck = y.value_counts()
ycheck

0    3653
1    1576
Name: loan_status, dtype: int64

In [6]:
X=df.drop('loan_status',axis=1)
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77524,16900.0,10.302,77500,0.612903,10,2,47500
77526,18300.0,10.895,83100,0.638989,11,2,53100
77528,15100.0,9.557,70500,0.574468,9,2,40500
77531,19100.0,11.261,86600,0.653580,12,2,56600


In [7]:
# Split the data into X_train, X_test, y_train, y_test
# the ratio of loan_status of 0 & 1 is unbalanced.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, stratify=y, random_state=19)
unique_element_train, counts_elements_train = np.unique(y_train, return_counts=True)
unique_element_test, counts_elements_test = np.unique(y_test, return_counts=True)
print(np.asarray((unique_element_train, counts_elements_train)))
print(np.asarray((unique_element_test, counts_elements_test)))

[[   0    1]
 [1826  788]]
[[   0    1]
 [1827  788]]


In [8]:
Scaler = StandardScaler().fit(X_train)
X_train_scaled = Scaler.transform(X_train)
X_test_scaled = Scaler.transform(X_test)

## Create, Fit and Compare Models

Create a Logistic Regression model, fit it to the data, and print the model's score. Do the same for a Random Forest Classifier. You may choose any starting hyperparameters you like. 

Which model performed better? How does that compare to your prediction? Write down your results and thoughts in the designated markdown cell.

### <span style='font-family:Helvetica;color:darkorange'>Logistic Regression

##### <span style='font-family:Helvetica;color:#F5B041 '>Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_scaled,y_train)

print('Model: Logistic Regression')

print(f'Test Score: {model.score(X_train_scaled, y_train)}')

print(f'Train Score: {model.score(X_test_scaled, y_test)}')

Model: Logistic Regression
Test Score: 0.9200459066564652
Train Score: 0.918546845124283


In [16]:
# Print the classification report for the Logistic Regression model

y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1827
           1       0.81      0.96      0.88       788

    accuracy                           0.92      2615
   macro avg       0.89      0.93      0.91      2615
weighted avg       0.93      0.92      0.92      2615



### <span style='font-family:Helvetica;color:darkorange'> Random Forest Classifier

In [12]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 3)
clf = classifier.fit(X_train_scaled,y_train)
print('Model: Random Forest Classifier')
print(f'Test Score: {clf.score(X_train_scaled, y_train)}')
print(f'Train Score: {clf.score(X_test_scaled, y_test)}')
 

Model: RandomForestClassifier
Test Score: 0.9812547819433818
Train Score: 0.8573613766730401


In [13]:
# Print the classification report for the Random Forest Classifier model

y_pred_RFC = clf.predict(X_test_scaled)

print(classification_report(y_test, y_pred_RFC))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1827
           1       0.77      0.75      0.76       788

    accuracy                           0.86      2615
   macro avg       0.83      0.83      0.83      2615
weighted avg       0.86      0.86      0.86      2615



## Summary


Compared to two results from two different classifiers, the output is the same as my prediction.   
The Logistic Regression model had a higher accuracy score 0.92 while the Random Forest Classifier model had 0.86.